In [40]:
from qiskit import IBMQ
from qiskit.opflow.primitive_ops import PauliOp, PauliSumOp
from qiskit.aqua.operators.legacy import WeightedPauliOperator           
from qiskit.quantum_info import Pauli
from qiskit.algorithms import VQE
import numpy as np

In [2]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic') 

In [3]:
backend = provider.get_backend('ibmq_lima')
defaults = backend.defaults()

In [92]:
conexions = [ indx for indx in defaults.instruction_schedule_map.qubits_with_instruction('cx') if indx[0]<indx[1]  ]
conexions

[(0, 1), (1, 2), (1, 3), (3, 4)]

In [74]:
#Hamiltonian 
def HeisenbergHamiltonian( J=1, H=1, num_qubits=2, neighbours=[(0,1)] ):
    """
    Qiskit operator of the 3-D Heisemberg Hamiltonian of a lattice of spins.
    
    H = - J Σ_j ( X_j X_{j+1} + Y_j Y_{j+1} + Z_j Z_{j+1} ) - H Σ_j Z_j
    
    input:
        J          : Real. Coupling constant.
        H          : Real. External magnetic field.
        num_qubits : Integer. Number of qubits.
        neighbours : List of tuples. Coupling between the spins.
    output:
        Hamiltonian : BaseOperator of Qiskit. Heisenberg Hamiltonian of the system.
    """
    num_op = num_qubits + 3*len(neighbours)
    Hamiltonian_op_x = []    
    Hamiltonian_op_z = []  
    Hamiltonian_coef = num_qubits*[-H] + num_op*[-J]
    
    for idx in range(num_qubits):
        op_x = np.zeros( num_qubits )
        op_z = np.zeros( num_qubits )
        op_z[idx] = 1
        Hamiltonian_op_x.append( op_x.copy() )
        Hamiltonian_op_z.append( op_z.copy() )        
    
    for idx in neighbours:
        op_x = np.zeros( num_qubits )
        op_z = np.zeros( num_qubits )
        op_x[idx[0]] = 1
        op_x[idx[1]] = 1
        Hamiltonian_op_x.append( op_x.copy() )
        Hamiltonian_op_z.append( op_z.copy() )
        op_z[idx[0]] = 1
        op_z[idx[1]] = 1
        Hamiltonian_op_x.append( op_x.copy() )
        Hamiltonian_op_z.append( op_z.copy() )        
        op_x[idx[0]] = 0
        op_x[idx[1]] = 0
        Hamiltonian_op_x.append( op_x.copy() )
        Hamiltonian_op_z.append( op_z.copy() )        
        
    Hamiltonian = WeightedPauliOperator( 
        [ [Hamiltonian_coef[j], Pauli( ( Hamiltonian_op_z[j], Hamiltonian_op_x[j] )) ] 
         for j in range(num_op) ] )
    return Hamiltonian

QubitOp = HeisenbergHamiltonian( num_qubits=4, neighbours=[(0,1),(1,2),(2,3),(3,0)] )

In [75]:
print( QubitOp.print_details() )

IIIZ	(1+0j)
IIZI	(1+0j)
IZII	(1+0j)
ZIII	(1+0j)
IIXX	(1+0j)
IIYY	(1+0j)
IIZZ	(1+0j)
IXXI	(1+0j)
IYYI	(1+0j)
IZZI	(1+0j)
XXII	(1+0j)
YYII	(1+0j)
ZZII	(1+0j)
XIIX	(1+0j)
YIIY	(1+0j)
ZIIZ	(1+0j)



In [84]:
def Label2Chain(QubitOp):
    """
    Transform a string of Pauli matrices into a numpy array.
    'I' --> 0
    'X' --> 1
    'Y' --> 2
    'Z' --> 3
    
    input:
        QubitOp : BaseOperator of Qiskit.
    output:
        ops     : narray of the Pauli operators.
                  ops.shape = ( number_of_operators, number_of_qubits )
        coef    : coefficient of each Pauli operator.
    """
    Dict = {'I' : 0,
           'X' : 1,
           'Y' : 2,
           'Z' : 3}
    ops = [[ Dict.get(idx2.to_label()) for idx2 in idx[1]] for idx in QubitOp.paulis ]
    coef = [ idx[0] for idx in QubitOp.paulis ]
    return ops, coef

In [85]:
Label2Chain(QubitOp)

([[3, 0, 0, 0],
  [0, 3, 0, 0],
  [0, 0, 3, 0],
  [0, 0, 0, 3],
  [1, 1, 0, 0],
  [2, 2, 0, 0],
  [3, 3, 0, 0],
  [0, 1, 1, 0],
  [0, 2, 2, 0],
  [0, 3, 3, 0],
  [0, 0, 1, 1],
  [0, 0, 2, 2],
  [0, 0, 3, 3],
  [1, 0, 0, 1],
  [2, 0, 0, 2],
  [3, 0, 0, 3]],
 [(1+0j),
  (1+0j),
  (1+0j),
  (1+0j),
  (1+0j),
  (1+0j),
  (1+0j),
  (1+0j),
  (1+0j),
  (1+0j),
  (1+0j),
  (1+0j),
  (1+0j),
  (1+0j),
  (1+0j),
  (1+0j)])